In [3]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [4]:
### load the meta data

data = []
with gzip.open('meta_Software.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

26790
{'category': [], 'tech1': '', 'description': [], 'fit': '', 'title': 'HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK CONCEPTS', 'also_buy': [], 'tech2': '', 'brand': 'HOLT. RINEHART AND WINSTON', 'feature': [], 'rank': '25,550 in Software (', 'also_view': [], 'main_cat': 'Software', 'similar_item': '', 'date': '</div>', 'price': '.a-box-inner{background-color:#fff}#alohaBuyBoxWidget .selected{background-color:#fffbf3;border-color:#e77600;box-shadow:0 0 3px rgba(228,121,17,.5)}#alohaBuyBoxWidget .contract-not-available{color:gray}#aloha-cart-popover .aloha-cart{height:auto;overflow:hidden}#aloha-cart-popover #aloha-cartInfo{float:left}#aloha-cart-popover #aloha-cart-details{float:right;margin-top:1em}#aloha-cart-popover .deviceContainer{width:160px;float:left;padding-right:10px;border-right:1px solid #ddd}#aloha-cart-popover li:last-child{border-right:0}#aloha-cart-popover .aloha-device-title{height:3em;overflow:hidden}#aloha-cart-popover .aloha-thumbnail-container{height:100px;mar

In [5]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

26790


In [6]:
### remove rows with unformatted title (i.e. some 'title' may still contain html style content)

df3 = df.fillna('')
df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
print(len(df4))
print(len(df5))

0
26790


In [7]:
df5.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,[],,[],,HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,[],,HOLT. RINEHART AND WINSTON,[],"25,550 in Software (",[],Software,,</div>,.a-box-inner{background-color:#fff}#alohaBuyBo...,0030672120,[],[],
1,[],,"[, <b>Latin rhythms that will get your kids si...",,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",[],,McGraw Hill,[],"15,792 in Software (",[],Software,,</div>,,0071480935,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,
2,[],,[<b>Connect is the only integrated learning sy...,,Connect with LearnSmart Access Card for Microb...,[],,McGraw-Hill Science/Engineering/Math,[],"16,900 in Software (",[],Software,,</div>,,007329506X,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,
3,[],,[],,LearnSmart Standalone Access Card for Prescott...,[],,McGraw-Hill Education,[],"12,986 in Software (",[],Software,,</div>,,0073513458,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,
4,[],,[<i>Anatomy &amp; Physiology Revealed Cat</i> ...,,Anatomy &amp; Physiology Revealed Student Acce...,"[0323394612, 0323227937, 1118527488]",,McGraw-Hill Education,[],"14,861 in Software (",[],Software,,</div>,$4.83,0073525758,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,


In [8]:
empty_categories = df5['category'].apply(lambda x: len(x) == 0).sum()
software_categories = df5['category'].apply(lambda x: 'Software' in x).sum()

print("Number of empty categories:", empty_categories)
print("Number of categories labeled as 'Software':", software_categories)


Number of empty categories: 2063
Number of categories labeled as 'Software': 24727


In [9]:
empty_prices = df5['price'].isna().sum()
software_prices = df5['price'].apply(lambda x: 'Software' in str(x)).sum()

print("Number of empty prices:", empty_prices)
print("Number of prices labeled as 'Software':", software_prices)


Number of empty prices: 0
Number of prices labeled as 'Software': 0


In [10]:
duplicate_asins = df5.duplicated(subset='asin').sum()
print("Number of duplicate ASINs:", duplicate_asins)


Number of duplicate ASINs: 5151


In [11]:
empty_descriptions = df5['description'].apply(lambda x: len(x) == 0).sum()
print("Number of empty descriptions:", empty_descriptions)


Number of empty descriptions: 3595


In [12]:
software_categories = df5['main_cat'].apply(lambda x: 'Software' in x).sum()
print("Number of categories labeled as 'Software':", software_categories)


Number of categories labeled as 'Software': 23771


In [14]:
non_div_date = df5[df5['date'] != '</div>'].shape[0]
print("Number of rows with date not equal to '</div>':", non_div_date)


Number of rows with date not equal to '</div>': 3817


In [27]:
filtered_df = df5[df5['main_cat'].apply(lambda x: 'Software' in x)]
filtered_df = filtered_df.drop_duplicates(subset='asin')
filtered_df = filtered_df[filtered_df['category'].apply(lambda x: 'Software' in x)]
filtered_df = filtered_df.drop(['imageURL', 'imageURLHighRes'], axis=1)

# Print the resulting DataFrame
print(filtered_df.shape)
filtered_df.head()


(17424, 17)


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
11,"[Software, Education &amp; Reference]",,"[Slides with Video, Teaching Public Speaking O...",,Instructor's Resource CD-ROM for The Art of Sp...,[],,McGraw Hill,[],"18,178 in Software (",[],Software,,</div>,$8.00,007742817X,
55,"[Software, Education &amp; Reference]",,"[Contains a guided tour of the program, Planni...",,Magruder's American Government Resource Pro CD...,[],,Magruder's,[],"19,702 in Software (",[0130679550],Software,,</div>,,0130438480,
57,"[Software, Education & Reference, Test Prepara...",,[],,Prentice Hall Test Manager a Comprehensive Sui...,[],,prentice hall,[],"54,036 in Software (",[],Software,,</div>,,0130852414,
59,"[Software, Education &amp; Reference, Test Pre...",,"[Windos 95, 98, NT4, 200, XP\nMac OS 9.1-9.2 O...",,Magruder's American Government Itext Interacti...,[],,Magruder's,[Interactive Learning Tools-Bring Content to l...,"52,031 in Software (",[],Software,,</div>,,0131817949,
68,"[Software, Design &amp; Illustration, CAD]",,"[2.5 Floppy, , ]",,AUTOCAD The Student Edition Release 10 (1982-89),[],,Autodesk,[],"30,901 in Software (",[],Software,,</div>,,0201656302,


In [16]:
non_empty_also_buy = filtered_df['also_buy'].apply(lambda x: len(x) > 0).sum()
print("Number of rows with non-empty 'also_buy':", non_empty_also_buy)


Number of rows with non-empty 'also_buy': 1701


In [17]:
non_empty_similar_item = filtered_df['similar_item'].apply(lambda x: len(x) > 0).sum()
print("Number of rows with non-empty 'similar_item':", non_empty_similar_item)


Number of rows with non-empty 'similar_item': 0


In [18]:
non_empty_similar_item = filtered_df['asin'].apply(lambda x: len(x) > 0).sum()
print("Number of rows with non-empty 'asin':", non_empty_similar_item)


Number of rows with non-empty 'asin': 17424


In [19]:
filtered_df.to_csv('filtered_data.csv', index=False)


In [20]:
#load the data
data = []
with gzip.open('Software.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

In [30]:
reviews = pd.DataFrame.from_dict(data)


In [22]:
reviews.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN
1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN
2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,{'Format:': ' Loose Leaf'},Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200,7,NaN
3,3.0,True,"02 17, 2014",APRDVZ6QBIQXT,0077613252,{'Format:': ' Loose Leaf'},Lucy,This book was missing pages!!! Important pages...,missing pages!!,1392595200,3,NaN
4,5.0,False,"10 14, 2013",A2JZTTBSLS1QXV,0077775473,NaN,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,1381708800,NaN,NaN


In [31]:
reviews = reviews.drop(['unixReviewTime', 'image','style'], axis=1)
filtered_reviews = reviews[reviews['asin'].isin(filtered_df['asin'])]

filtered_reviews.shape
filtered_reviews.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote
93,5.0,False,"07 23, 2008",A8IOST6U6WH9B,0615179088,C. Radey,Human Japanese is a truly superb introduction ...,Human Japanese,12
94,5.0,False,"06 4, 2008",A1MUV9F35OROS5,0615179088,D. Abel,I got Human Japanese as a demo from its websit...,Best Japanese Program Available,11
95,4.0,False,"04 8, 2008",A27PAMABWVQ892,0615179088,piepiepie75,My first experience with Human Japanese was th...,Better than the Human Japanese 1...but not muc...,99
96,5.0,False,"03 26, 2008",A3HWWVK0L3JEKF,0615179088,K. Grier,This is the first language software that I hav...,Great Product,4
97,5.0,False,"02 20, 2008",A3NO2V2JU4Y8UY,0615179088,H. Granat,Human japanese is the best pc program for lear...,Love it!,2


In [32]:
filtered_reviews.to_csv('reviews_full.csv', index=False)
